In [1]:
import pandas as pd
from pathlib import Path
import pretty_midi as pm
import music21 as m21
import numpy as np

In [2]:
BASE_PATH = Path("../")

In [3]:
quant_df = pd.read_excel(Path(BASE_PATH,"dataframes","quant_cleaning_info.xlsx"),index_col=0)
quant_df["scorexml_path"] = quant_df["opus"] + "/musicxml_cleaned.musicxml"
quant_df["scoremidi_path"] = quant_df["opus"] + "/midi_cleaned.mid"

# First we check if the annotations are in the correct format

- check if they are only db and b and bR

In [78]:
#check if only db,b,bR are in the annotations
def check_type(row):
    try:
        ann_df = pd.read_csv(Path(BASE_PATH,row.opus,"ann_quant_cleaned.txt"),header=None, names=["time","time2","type"],sep='\t')
        allowed_types = ["db","b","bR"]
        uncorrect_list = [ (at["type"],at["time"]) for i,at in ann_df.iterrows() if at["type"] not in allowed_types]
        if len(uncorrect_list)!=0:
            print("Wrong type for piece", row.opus)
            print(uncorrect_list)
    except Exception as e:
        print("Problems loading for opus", row.opus)
        print(e)

for i,row in  quant_df.iterrows():  
    check_type(row)

Problems loading for opus Beethoven/Piano_Sonatas/32-2
[Errno 2] File ..\Beethoven\Piano_Sonatas\32-2\ann_quant_cleaned.txt does not exist: '..\\Beethoven\\Piano_Sonatas\\32-2\\ann_quant_cleaned.txt'


# If in correct format, import them

In [79]:
#load the beat position
def load_anns_from_file(row):
    try:
        ann_df = pd.read_csv(Path(BASE_PATH,row.opus,"ann_quant_cleaned.txt"),header=None, names=["time","time2","type"],sep='\t')
#         beat_list = list(ann_df["time"])
#         downbeat_list = list(ann_df[ann_df["type"]=="db"]["time"])
        ann_list = [(row["time"],row["type"]) for i,row in ann_df.iterrows()]
        return ann_list
    except Exception as e:
        print("Problems for opus", row.opus, e)
        return np.nan

quant_df['beats'] = quant_df.apply(load_anns_from_file, axis=1)

Problems for opus Beethoven/Piano_Sonatas/32-2 [Errno 2] File ..\Beethoven\Piano_Sonatas\32-2\ann_quant_cleaned.txt does not exist: '..\\Beethoven\\Piano_Sonatas\\32-2\\ann_quant_cleaned.txt'


# Let's compute some more features that we need to check the quality
- Add the time signature changes, aligning it between the score and the midi

In [101]:
def score2ts_changes(score_path, remove_duplicates = True):
    score = m21.converter.parse(score_path)
    ts_changes = [ str(ts.numerator)+"/"+str(ts.denominator) for ts in score.parts[0].recurse().getElementsByClass(m21.meter.TimeSignature)]
    if remove_duplicates:
        #remove the duplicates
        ts_changes = [ts for i,ts in enumerate(ts_changes) if i==0 or (i!=0 and ts_changes[i-1]!=ts)]
    return ts_changes

def midi2ts_changes(midi_path, remove_duplicates = True):
    midi = pm.PrettyMIDI(midi_path)
    ts_changes = [(ts.time, str(ts.numerator)+"/"+str(ts.denominator)) for ts in midi.time_signature_changes]
    if remove_duplicates:
    #remove the duplicates
        ts_changes = [ts for i,ts in enumerate(ts_changes) if i==0 or (i!=0 and ts_changes[i-1][1]!=ts[1])]
    return ts_changes

def align_score_ts_with_midi_ts(score_path,midi_path):
    midi_ts_changes = midi2ts_changes(midi_path)
    score_ts_changes = score2ts_changes(score_path)
    out = []
    for si, sts in enumerate(score_ts_changes):
        found_at_index = None
        for mi, mts in enumerate(midi_ts_changes):
            if sts == mts[1]:
                out.append((sts,mts[0]))
                found_at_index = mi
                break
        if found_at_index is None:
            raise(ValueError("The algorithm of ts aligning is not working"))
        if found_at_index< len(midi_ts_changes):
            midi_ts_changes = midi_ts_changes[found_at_index+1:]
    return out

In [107]:
def row_to_ts_align(row):
    try:
        return align_score_ts_with_midi_ts(str(Path(BASE_PATH,row["scorexml_path"])),str(Path(BASE_PATH,row["scoremidi_path"])))
    except Exception as e:
        print("Processing piece",row["opus"], "..................Exception!!" )
        print(e)
        return None 
    
quant_df["ts_changes"] = quant_df.apply(row_to_ts_align, axis = 1)

Processing piece Beethoven/Piano_Sonatas/32-2 ..................Exception!!
Error in getting DynamicWedges...Measure no. 57 P1


In [108]:
quant_df

,opus,done,multiple_time_signatures,big_tempo_changes,time_signatures,problems,other,invalid_nak_alignment,scorexml_path,scoremidi_path,beats,ts_changes
0,Bach/Fugue/bwv_846,True,False,False,4/4,last db missing,NaN,0.0,Bach/Fugue/bwv_846/musicxml_cleaned.musicxml,Bach/Fugue/bwv_846/midi_cleaned.mid,"[(0.5, b), (1.0, b), (1.5, b), (2.0, db), (2.5...","[(4/4, 0.0)]"
1,Bach/Fugue/bwv_848,True,False,False,4/4,last db missing,NaN,0.0,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Bach/Fugue/bwv_848/midi_cleaned.mid,"[(1.0, b), (1.5, b), (2.0, db), (2.5, b), (3.0...","[(4/4, 0.0)]"
2,Bach/Fugue/bwv_854,True,False,False,4/4,NaN,NaN,0.0,Bach/Fugue/bwv_854/musicxml_cleaned.musicxml,Bach/Fugue/bwv_854/midi_cleaned.mid,"[(1.0, b), (1.5, b), (2.0, db), (2.5, b), (3.0...","[(4/4, 0.0)]"
3,Bach/Fugue/bwv_856,True,False,False,3/8,NaN,NaN,0.0,Bach/Fugue/bwv_856/musicxml_cleaned.musicxml,Bach/Fugue/bwv_856/midi_cleaned.mid,"[(0.0, b), (0.25, db), (0.5, b), (0.75, b), (1...","[(3/8, 0.25)]"
4,Bach/Fugue/bwv_857,True,False,False,4/4,NaN,NaN,0.0,Bach/Fugue/bwv_857/musicxml_cleaned.musicxml,Bach/Fugue/bwv_857/midi_cleaned.mid,"[(0.5, b), (1.0, b), (1.5, b), (2.0, db), (2.5...","[(4/4, 0.0)]"
...,...,...,...,...,...,...,...,...,...,...,...,...
232,Schumann/Kreisleriana/7,True,False,False,2/4,NaN,NaN,0.0,Schumann/Kreisleriana/7/musicxml_cleaned.musicxml,Schumann/Kreisleriana/7/midi_cleaned.mid,"[(0.125, db), (0.625, b), (1.125, db), (1.625,...","[(2/4, 0.125)]"
233,Schumann/Toccata,True,False,False,2/4,no pb,NaN,0.0,Schumann/Toccata/musicxml_cleaned.musicxml,Schumann/Toccata/midi_cleaned.mid,"[(0.0, db), (0.491804, b), (0.983608, db), (1....","[(2/4, 0.0)]"
234,Schumann/Toccata_repeat,True,False,False,2/4,NaN,NaN,0.0,Schumann/Toccata_repeat/musicxml_cleaned.musicxml,Schumann/Toccata_repeat/midi_cleaned.mid,"[(0.0, db), (0.491804, b), (0.983608, db), (1....","[(2/4, 0.0)]"
235,Scriabin/Etudes_op_8/11,True,False,False,3/4,"the beat were like in 4/4, the db were correct",i guess the old midi_cleaned was produced from...,0.0,Scriabin/Etudes_op_8/11/musicxml_cleaned.musicxml,Scriabin/Etudes_op_8/11/midi_cleaned.mid,"[(0.0, db), (1.333333, b), (2.666666, b), (3.9...","[(3/4, 0.0)]"


- move the beats/downbeats tu the leftmost onset in the 35ms window

In [109]:
# move the manual annotations to the closest leftmost note in a 35ms window

def align_ann_to_closest_leftmost_event(row):
    print("Aligning opus", row.opus)
    midi_path = Path(BASE_PATH,row["opus"],"midi_cleaned.mid")
    midi = pm.PrettyMIDI(str(midi_path))
    
#     if row.opus == "Beethoven/Piano_Sonatas/32-2":
#         return np.nan
    if (type(row.beats) == float and pd.isnull(row.beats)):
        print("Beat or downbeat midding, alignment skipped")
        return np.nan
    
    # align annotations
    aligned_beats = []
    for b_time, b_type in row.beats:
        # find notes in the window
        close_notes_onset = [note.start for inst in midi.instruments for note in inst.notes 
                             if (note.start >= b_time - 0.0175) and (note.start <= b_time + 0.0175)]
        if len(close_notes_onset) == 0: # if no close note found (beat on a rest)
            aligned_beats.append((b_time,b_type))
        else:
            aligned_beats.append((sorted(close_notes_onset)[0],b_type)) # align to the leftmost of the window
            shift = np.abs(b_time- sorted(close_notes_onset)[0])
            if shift > 0:
                print("beat at",b_time, "moved of ",shift)
                
    return aligned_beats

In [110]:
quant_df['beats'] = quant_df.apply(align_ann_to_closest_leftmost_event, axis=1)

Aligning opus Bach/Fugue/bwv_846
Aligning opus Bach/Fugue/bwv_848
Aligning opus Bach/Fugue/bwv_854
Aligning opus Bach/Fugue/bwv_856
beat at 14.75 moved of  0.0010416666666674956
beat at 15.0 moved of  0.0010416666666674956
beat at 22.25 moved of  0.0010416666666657193
beat at 22.5 moved of  0.0010416666666657193
beat at 53.500013 moved of  1.3000000002705292e-05
Aligning opus Bach/Fugue/bwv_857


C:\Users\fosca\.conda\envs\mir_research\lib\site-packages\pretty_midi\pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Aligning opus Bach/Fugue/bwv_858
Aligning opus Bach/Fugue/bwv_860
beat at 128.250024 moved of  2.3999999996249244e-05
Aligning opus Bach/Fugue/bwv_862
Aligning opus Bach/Fugue/bwv_863
Aligning opus Bach/Fugue/bwv_864
Aligning opus Bach/Fugue/bwv_865
Aligning opus Bach/Fugue/bwv_866
Aligning opus Bach/Fugue/bwv_867
Aligning opus Bach/Fugue/bwv_868
Aligning opus Bach/Fugue/bwv_870
Aligning opus Bach/Fugue/bwv_873
beat at 104.999998 moved of  1.9999999949504854e-06
Aligning opus Bach/Fugue/bwv_874
beat at 99.000002 moved of  1.9999999949504854e-06
Aligning opus Bach/Fugue/bwv_875
Aligning opus Bach/Fugue/bwv_876
Aligning opus Bach/Fugue/bwv_880
Aligning opus Bach/Fugue/bwv_883
Aligning opus Bach/Fugue/bwv_884
beat at 37.0 moved of  0.0010416666666657193
beat at 37.25 moved of  0.0010416666666657193
beat at 38.5 moved of  0.0010416666666657193
beat at 38.75 moved of  0.0010416666666657193
Aligning opus Bach/Fugue/bwv_885
beat at 124.500001 moved of  9.999999974752427e-07
Aligning opus Bach

Aligning opus Bach/Prelude/bwv_864
Aligning opus Bach/Prelude/bwv_865
Aligning opus Bach/Prelude/bwv_866
Aligning opus Bach/Prelude/bwv_867
Aligning opus Bach/Prelude/bwv_868
Aligning opus Bach/Prelude/bwv_870
Aligning opus Bach/Prelude/bwv_873
beat at 137.258254 moved of  0.008253999999993766
Aligning opus Bach/Prelude/bwv_874
beat at 167.249987 moved of  1.2999999995599865e-05
Aligning opus Bach/Prelude/bwv_875
Aligning opus Bach/Prelude/bwv_876
Aligning opus Bach/Prelude/bwv_880
Aligning opus Bach/Prelude/bwv_883
Aligning opus Bach/Prelude/bwv_884
Aligning opus Bach/Prelude/bwv_885
beat at 1.4285709999999998 moved of  2.220446049250313e-16
beat at 2.8571419999999996 moved of  4.440892098500626e-16
beat at 5.714283999999999 moved of  8.881784197001252e-16
beat at 9.999997 moved of  1.7763568394002505e-15
beat at 11.428567999999999 moved of  1.7763568394002505e-15
beat at 19.999994 moved of  3.552713678800501e-15
beat at 22.857135999999997 moved of  3.552713678800501e-15
beat at 24.28

beat at 462.499994 moved of  5.999999984851456e-06
Aligning opus Beethoven/Piano_Sonatas/1-1
beat at 0.25861999999999996 moved of  1.0000000000287557e-06
beat at 0.77586 moved of  2.9999999999752447e-06
beat at 1.2931 moved of  5.000000000032756e-06
beat at 1.81034 moved of  7.000000000090267e-06
beat at 2.3275799999999998 moved of  9.000000000369823e-06
beat at 2.84482 moved of  1.100000000020529e-05
beat at 3.36206 moved of  1.3000000000040757e-05
beat at 3.8793 moved of  1.4999999999876223e-05
beat at 4.39654 moved of  1.700000000059987e-05
beat at 4.91378 moved of  1.8999999999991246e-05
beat at 5.431019999999999 moved of  2.100000000115898e-05
beat at 5.948259999999999 moved of  2.3000000000550358e-05
beat at 6.4655 moved of  2.5000000000829914e-05
beat at 6.98274 moved of  2.700000000022129e-05
beat at 7.49998 moved of  2.9000000000500847e-05
beat at 8.534460000000001 moved of  3.2999999998395424e-05
beat at 9.0517 moved of  3.500000000045134e-05
beat at 9.56894 moved of  3.70000

...

beat at 109.18048799999998 moved of  2.842170943040401e-14
beat at 109.672292 moved of  1.4210854715202004e-14
beat at 110.164096 moved of  1.4210854715202004e-14
beat at 110.6559 moved of  1.4210854715202004e-14
beat at 111.147704 moved of  1.4210854715202004e-14
beat at 111.639508 moved of  1.4210854715202004e-14
beat at 112.623116 moved of  1.4210854715202004e-14
beat at 113.11492 moved of  1.4210854715202004e-14
beat at 113.606724 moved of  1.4210854715202004e-14
beat at 114.098528 moved of  1.4210854715202004e-14
beat at 114.590332 moved of  1.4210854715202004e-14
beat at 115.082136 moved of  1.4210854715202004e-14
beat at 115.57394 moved of  1.4210854715202004e-14
beat at 116.065744 moved of  1.4210854715202004e-14
beat at 116.557548 moved of  1.4210854715202004e-14
beat at 117.049352 moved of  1.4210854715202004e-14
beat at 117.541156 moved of  1.4210854715202004e-14
beat at 118.03296 moved of  1.4210854715202004e-14
beat at 118.52476399999999 moved of  2.842170943040401e-14
bea

beat at 291.639772 moved of  5.684341886080802e-14
beat at 292.131576 moved of  5.684341886080802e-14
beat at 292.62338 moved of  5.684341886080802e-14
beat at 293.115184 moved of  5.684341886080802e-14
beat at 293.606988 moved of  5.684341886080802e-14
beat at 294.098792 moved of  5.684341886080802e-14
beat at 294.590596 moved of  5.684341886080802e-14
beat at 295.0824 moved of  5.684341886080802e-14
beat at 298.52502799999996 moved of  5.684341886080802e-14
beat at 299.01683199999997 moved of  5.684341886080802e-14
beat at 299.50863599999997 moved of  5.684341886080802e-14
beat at 300.49224399999997 moved of  5.684341886080802e-14
beat at 300.984048 moved of  5.684341886080802e-14
beat at 301.475852 moved of  5.684341886080802e-14
beat at 304.426676 moved of  5.684341886080802e-14
beat at 304.91848 moved of  5.684341886080802e-14
beat at 305.410284 moved of  5.684341886080802e-14
beat at 305.902088 moved of  5.684341886080802e-14
beat at 306.393892 moved of  5.684341886080802e-14
bea

beat at 1.1842110000000001 moved of  2.220446049250313e-16
beat at 1.973685 moved of  2.220446049250313e-16
beat at 2.3684220000000002 moved of  4.440892098500626e-16
beat at 3.94737 moved of  4.440892098500626e-16
beat at 4.7368440000000005 moved of  8.881784197001252e-16
beat at 6.710528999999999 moved of  8.881784197001252e-16
beat at 7.5000029999999995 moved of  8.881784197001252e-16
beat at 7.89474 moved of  8.881784197001252e-16
beat at 9.473688000000001 moved of  1.7763568394002505e-15
beat at 9.671056 moved of  5.000000005139782e-07
beat at 10.46053 moved of  5.000000005139782e-07
beat at 10.855267 moved of  5.000000005139782e-07
beat at 11.250003999999999 moved of  5.000000005139782e-07
beat at 11.447372999999999 moved of  1.7763568394002505e-15
beat at 11.644741 moved of  5.000000005139782e-07
beat at 12.434215 moved of  5.000000005139782e-07
beat at 12.828952000000001 moved of  4.999999987376214e-07
beat at 13.223689000000002 moved of  4.999999987376214e-07
beat at 13.421057

beat at 160.26322199999998 moved of  2.842170943040401e-14
beat at 160.85532800000001 moved of  4.999999987376214e-07
beat at 161.44743300000002 moved of  2.842170943040401e-14
beat at 162.039539 moved of  4.999999987376214e-07
beat at 163.22375 moved of  4.999999987376214e-07
beat at 164.407961 moved of  4.999999987376214e-07
beat at 165.592172 moved of  4.999999987376214e-07
beat at 166.776383 moved of  4.999999987376214e-07
beat at 167.36848799999999 moved of  2.842170943040401e-14
beat at 167.960594 moved of  4.999999703159119e-07
beat at 169.144805 moved of  4.999999987376214e-07
beat at 169.73691000000002 moved of  2.842170943040401e-14
beat at 170.329015 moved of  4.999999987376214e-07
beat at 171.513227 moved of  4.999999987376214e-07
beat at 172.69743799999998 moved of  4.999999703159119e-07
beat at 173.28954299999998 moved of  2.842170943040401e-14
beat at 173.881649 moved of  4.999999987376214e-07
beat at 175.06586000000001 moved of  4.999999987376214e-07
beat at 175.460597 

beat at 353.88172099999997 moved of  4.999999418942025e-07
beat at 354.473826 moved of  5.684341886080802e-14
beat at 355.065932 moved of  4.999999987376214e-07
beat at 355.65803700000004 moved of  5.684341886080802e-14
beat at 356.25014300000004 moved of  5.000000555810402e-07
beat at 357.434354 moved of  4.999999987376214e-07
beat at 358.618565 moved of  4.999999987376214e-07
beat at 359.802776 moved of  4.999999987376214e-07
beat at 360.986987 moved of  4.999999987376214e-07
beat at 362.171198 moved of  4.999999987376214e-07
beat at 363.355409 moved of  4.999999987376214e-07
beat at 364.53962 moved of  4.999999987376214e-07
beat at 365.723831 moved of  4.999999987376214e-07
beat at 366.31593599999997 moved of  5.684341886080802e-14
beat at 366.90804199999997 moved of  4.999999418942025e-07
beat at 368.09225299999997 moved of  4.999999418942025e-07
beat at 369.27646400000003 moved of  5.000000555810402e-07
beat at 370.460675 moved of  4.999999987376214e-07
beat at 371.644886 moved of

- move the tim signature to a close beat/downbeat

In [111]:
def align_ts_to_beat(row):
    print("Aligning opus", row.opus)
    if (type(row.beats) == float and pd.isnull(row.beats)):
        print("Beat or downbeat missing, alignment skipped")
        return np.nan
    
    downbeats = [b[0] for b in row.beats if b[1]=="db"]
    
    # align ts
    aligned_ts = []
    for ts in row.ts_changes:
        # find beat in the window
        close_db = [db for db in downbeats if (db >= ts[1] - 0.0175) and (db <= ts[1] + 0.0175)]
        if ts[1] == 0: #initial time signature, can not coincide with downbeats, so don't align it
            aligned_ts.append(ts)
        elif len(close_db) != 1: # if no close note found (beat on a rest)
            print("Some problems for ts",ts)
        else:
            aligned_ts.append((ts[0],close_db[0])) # align to the close downbeat
            shift = np.abs(ts[1]- close_db[0])
            if shift > 0:
                print("ts",ts, "moved of ",shift)
    
    return aligned_ts


quant_df["ts_changes"] = quant_df.apply(align_ts_to_beat, axis=1)

Aligning opus Bach/Fugue/bwv_846
Aligning opus Bach/Fugue/bwv_848
Aligning opus Bach/Fugue/bwv_854
Aligning opus Bach/Fugue/bwv_856
Aligning opus Bach/Fugue/bwv_857
Aligning opus Bach/Fugue/bwv_858
Aligning opus Bach/Fugue/bwv_860
Aligning opus Bach/Fugue/bwv_862
Aligning opus Bach/Fugue/bwv_863
Aligning opus Bach/Fugue/bwv_864
Aligning opus Bach/Fugue/bwv_865
Aligning opus Bach/Fugue/bwv_866
Aligning opus Bach/Fugue/bwv_867
Aligning opus Bach/Fugue/bwv_868
Aligning opus Bach/Fugue/bwv_870
Aligning opus Bach/Fugue/bwv_873
Aligning opus Bach/Fugue/bwv_874
Aligning opus Bach/Fugue/bwv_875
Aligning opus Bach/Fugue/bwv_876
Aligning opus Bach/Fugue/bwv_880
Aligning opus Bach/Fugue/bwv_883
Aligning opus Bach/Fugue/bwv_884
Aligning opus Bach/Fugue/bwv_885
Aligning opus Bach/Fugue/bwv_887
Aligning opus Bach/Fugue/bwv_888
Aligning opus Bach/Fugue/bwv_889
Aligning opus Bach/Fugue/bwv_891
Aligning opus Bach/Fugue/bwv_892
Aligning opus Bach/Fugue/bwv_893
Aligning opus Bach/Italian_concerto
Alignin

In [11]:
quant_df

,opus,done,multiple_time_signatures,big_tempo_changes,time_signatures,problems,other,invalid_nak_alignment,scorexml_path,scoremidi_path,beats,ts_changes
0,Bach/Fugue/bwv_846,True,False,False,4/4,last db missing,NaN,0.0,Bach/Fugue/bwv_846/musicxml_cleaned.musicxml,Bach/Fugue/bwv_846/midi_cleaned.mid,"[(0.5, b), (1.0, b), (1.5, b), (2.0, db), (2.5...","[(4/4, 0.0)]"
1,Bach/Fugue/bwv_848,True,False,False,4/4,last db missing,NaN,0.0,Bach/Fugue/bwv_848/musicxml_cleaned.musicxml,Bach/Fugue/bwv_848/midi_cleaned.mid,"[(1.0, b), (1.5, b), (2.0, db), (2.5, b), (3.0...","[(4/4, 0.0)]"
2,Bach/Fugue/bwv_854,True,False,False,4/4,NaN,NaN,0.0,Bach/Fugue/bwv_854/musicxml_cleaned.musicxml,Bach/Fugue/bwv_854/midi_cleaned.mid,"[(1.0, b), (1.5, b), (2.0, db), (2.5, b), (3.0...","[(4/4, 0.0)]"
3,Bach/Fugue/bwv_856,True,False,False,3/8,NaN,NaN,0.0,Bach/Fugue/bwv_856/musicxml_cleaned.musicxml,Bach/Fugue/bwv_856/midi_cleaned.mid,"[(0.0, b), (0.25, db), (0.5, b), (0.75, b), (1...","[(3/8, 0.25)]"
4,Bach/Fugue/bwv_857,True,False,False,4/4,NaN,NaN,0.0,Bach/Fugue/bwv_857/musicxml_cleaned.musicxml,Bach/Fugue/bwv_857/midi_cleaned.mid,"[(0.5, b), (1.0, b), (1.5, b), (2.0, db), (2.5...","[(4/4, 0.0)]"
...,...,...,...,...,...,...,...,...,...,...,...,...
232,Schumann/Kreisleriana/7,True,False,False,2/4,NaN,NaN,0.0,Schumann/Kreisleriana/7/musicxml_cleaned.musicxml,Schumann/Kreisleriana/7/midi_cleaned.mid,"[(0.125, db), (0.625, b), (1.125, db), (1.625,...","[(2/4, 0.125)]"
233,Schumann/Toccata,True,False,False,2/4,no pb,NaN,0.0,Schumann/Toccata/musicxml_cleaned.musicxml,Schumann/Toccata/midi_cleaned.mid,"[(0.0, db), (0.491804, b), (0.9836080000000001...","[(2/4, 0.0)]"
234,Schumann/Toccata_repeat,True,False,False,2/4,NaN,NaN,0.0,Schumann/Toccata_repeat/musicxml_cleaned.musicxml,Schumann/Toccata_repeat/midi_cleaned.mid,"[(0.0, db), (0.491804, b), (0.9836080000000001...","[(2/4, 0.0)]"
235,Scriabin/Etudes_op_8/11,True,False,False,3/4,"the beat were like in 4/4, the db were correct",i guess the old midi_cleaned was produced from...,0.0,Scriabin/Etudes_op_8/11/musicxml_cleaned.musicxml,Scriabin/Etudes_op_8/11/midi_cleaned.mid,"[(0.0, db), (1.3333329999999999, b), (2.666665...","[(3/4, 0.0)]"


# Now compute quality factors

## Beats before the first note

In [12]:
def check_late_earl_annot(row):
    #load annotation file
    first_ann = row["beats"][0][0]
    last_ann =row["beats"][-1][0]
    #load midi file
    midi = pm.PrettyMIDI(str(Path(BASE_PATH,row.opus,"midi_cleaned.mid")))
    #extract the first note position
    note_ons = midi.get_onsets()
    first_ons = note_ons[0]
    last_ons = note_ons[-1]

    #check if the the first annotation is not before the first onset
    if first_ann < first_ons - 0.035: #35 ms accepted window
        raise ValueError("Wrong first beat at time", first_ann )
        
    #check if the the last annotation is not after the last onset
    if last_ann > last_ons + 0.035: #35 ms accepted window
        raise ValueError("Wrong last beat at time", last_ann )

        
for i,row in quant_df.iterrows():
    try:
        check_late_earl_annot(row)
    except Exception as e:
        print("Exception for",row["opus"])
        print(e)

C:\Users\fosca\.conda\envs\mir_research\lib\site-packages\pretty_midi\pretty_midi.py:101: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  RuntimeWarning)


Exception for Beethoven/Piano_Sonatas/32-2
'float' object is not subscriptable


## Beats downbeat ratio per time signature

In [112]:
def check_b_db_ratio(row):
    #load annotation file
    merged_annotations = row.beats
    #extract the numerator
    tss = row.ts_changes
    number_of_beats = [(ts2n_of_beats(ts[0]),ts[1]) for ts in tss]
    
    #check if the correct number of beat every downbeat
    counter = 1
    pickup = True
    rubato = False
    for ann in merged_annotations:  
        #set the eventual rubato flag
        if ann[1][-1]=="R":
            rubato = True
        #get the expected number of beats
        if not pickup: # get the left closest number of beats
            left_number_of_beats = [nob[0] for nob in number_of_beats if nob[1]<ann[0]]
            if len(left_number_of_beats) == 0: #case of multiple initial bars with a wrong durations
                actual_number == number_of_beats[0][0]
            else:
                actual_number = left_number_of_beats[-1]
        else: # get the first number of beats in the score
            actual_number = number_of_beats[0][0]
        #start checking the beats and downbeats
        if ann[1] == "db":
            if (not pickup) and (not rubato) and (counter != actual_number):
                print("Processing opus", row["opus"])
                print(tss)
                print("Wrong number of beats: (",counter, ") in ann",ann[0], ". Expecting", actual_number)
            pickup = False
            rubato = False
            counter = 1
        elif ann[1][0] == "b": #count "b" and "bR"
            counter += 1
        else:
            counter = 1
            print("Wrong annotation kind", ann[1], "in time",ann[0])

            
#helper function to have the number of beats for a given time signature
def ts2n_of_beats(ts):
    num = int(ts.split("/")[0])
    if num == 2 or num == 6: #duple meter
        return 2
    elif num == 3 or num == 9: #triple meter
        return 3
    elif num == 4 or num == 12: #quadruple meter
        return 4
    elif num == 24: #my choice
        return 8
    else: #complex meter
        return num
    
    
for i,row in quant_df.iterrows():
    try:
        check_b_db_ratio(row)
    except Exception as e:
        print("Failed for", row["opus"])
        print(e)

Failed for Beethoven/Piano_Sonatas/32-2
'float' object is not iterable
Processing opus Ravel/Miroirs/3_Une_Barque
[('2/4', 0.0), ('3/4', 29.999980000000004), ('2/4', 38.999974), ('3/4', 62.99995800000001), ('2/4', 71.99995200000001), ('1/4', 86.999942), ('3/4', 88.499941), ('4/4', 133.49991100000003), ('3/4', 139.499907), ('4/4', 143.99990400000002), ('3/4', 149.99990000000003), ('5/4', 154.49989700000003), ('2/4', 161.99989200000002), ('3/4', 194.99987000000002), ('2/4', 221.99985200000003), ('3/4', 242.99983800000004), ('4/4', 247.49983500000002), ('3/4', 253.49983100000003), ('4/4', 257.99982800000004), ('5/4', 269.99982000000006), ('2/4', 277.499815), ('3/4', 292.49980500000004), ('2/4', 296.99980200000005), ('3/4', 299.99980000000005), ('4/4', 304.49979700000006), ('3/4', 328.49978100000004), ('4/4', 341.99977200000006), ('3/4', 359.99976000000004), ('4/4', 373.499751), ('2/4', 385.499743), ('4/4', 388.49974100000003), ('3/4', 424.49971700000003), ('4/4', 437.99970800000006), ('3/

In [ ]:
quant_df[quan]

In [113]:
print(516//60)
print(516%60)

8
36


# First ts at the first db

In [119]:
def first_ts_at_first_db(row):
    try:
        first_db = [e[0] for e in row.beats if e[1]=='db'][0]
        first_ts = row.ts_changes[0][1]
        if first_db!= first_ts:
            print("Changed for piece",row.opus)
            new_tsc = row.ts_changes
            new_tsc[0] = (new_tsc[0][0],first_db)
            return new_tsc
        else:
            return row.ts_changes
    except:
        print("Not working for opus",row.opus)
        return None
        
quant_df["ts_change"] = quant_df.apply(first_ts_at_first_db,axis=1)

Not working for opus Beethoven/Piano_Sonatas/32-2


[5, 2, 3]

In [16]:
m21.converter.parse("C:\\Users\\fosca\\Desktop\\CNAM\\performed-midi-dataset\\Ravel\\Miroirs\\3_Une_Barque\\musicxml_cleaned.musicxml")

<music21.stream.Score 0x1c28ec07160>

In [99]:
scorepath = str(Path(BASE_PATH,"Bach/Fugue/bwv_884/musicxml_cleaned.musicxml"))
score_changes =score2ts_changes(scorepath,remove_duplicates=True)
print(score_changes)
print(len(score_changes))

['3/8']
1


In [102]:
def score2full_ts_changes(scorepath):
    scor = m21.converter.parse(scorepath)
    tsc= []
    actual_ts = None
    for i,m in enumerate(scor.parts[0].getElementsByClass('Measure')):
        dur = m.duration.quarterLength
        ts = m.timeSignature
        if ts is not None:
            actual_ts = ts
        #start chec
        if i== 0:
            tsc.append((dur, "{}/{}".format(actual_ts.numerator,actual_ts.denominator)))
        else:
            if dur!= tsc[-1][0] and dur!= 0: #don't know why, some measures have durations 0
                tsc.append((dur, "{}/{}".format(actual_ts.numerator,actual_ts.denominator)))
    return tsc

def ts2quarter_length(ts_string):
    parts = ts_string.split('/')
    return int(parts[0])*4/int(parts[1])

score2full_ts_changes(scorepath)
score2ts_changes(scorepath)

['3/8']

In [98]:
midipath = str(Path(BASE_PATH,"Bach/Fugue/bwv_884/midi_cleaned.mid"))
midi_changes =["{}m{}s,{}".format(int(e[0]//60),np.floor(e[0])%60,e[1]) for e in midi2ts_changes(midipath, remove_duplicates = True)]

print(len(midi_changes))
midi_changes

1


['0m0.0s,3/8']

In [104]:
def align_score_full_ts_with_midi_ts(score_path,midi_path):
    simple_changes = score2ts_changes(score_path)
    if len(simple_changes) == 1:
        out = [(0,simple_changes[0])]
    else:
        midi_ts_changes = midi2ts_changes(midi_path)
        score_ts_changes = score2full_ts_changes(score_path)
        out = []
        for si, (sql, sts) in enumerate(score_ts_changes):
            found_at_index = None
            for mi, mts in enumerate(midi_ts_changes):
                if sql == ts2quarter_length(mts[1]):
                    out.append((sts,mts[0]))
                    found_at_index = mi
                    break
            if found_at_index is None:
                raise(ValueError("The algorithm of ts aligning is not working"))
            if found_at_index< len(midi_ts_changes):
                midi_ts_changes = midi_ts_changes[found_at_index+1:]
        #now clean the output from the repetitions
        out = [(ts,time) for i, (ts,time)  in enumerate(out) if (i==0 or (i>0 and ts!= out[i-1][0]))]
    return out

align_score_full_ts_with_midi_ts(scorepath,midipath)

[(0, '3/8')]